In [ ]:
import pickle
import os
import numpy as np

In [ ]:
def read_data(path):
    data = []
    with open(path,"r") as infile:
        lines = infile.readlines()
        data = [ line.strip().split() for line in lines]
    return data

In [ ]:
def map_data(data,mapped_entity,mapped_relation):
    mapped_data=[]
    for line in data:
        mapped_data.append([mapped_entity[line[0]],mapped_relation[line[1]],mapped_entity[line[2]]])
    return mapped_data

In [ ]:
def read_entity_names(path):
    entity_names = {}
    with open(path,"r") as f:
        lines = f.readlines()
        for line in lines:
            content = line.split()
            if content[0] in entity_names:
                raise "Entity duplicated"
            entity_names[content[0]] = ' '.join(content[1:-2])
    return entity_names

In [ ]:
def read_pkl(filename):
    with open(filename,"rb") as f:
        pkl_dict = pickle.load(f)
    return pkl_dict

In [ ]:
def get_inverse_dict(mydict):
    inverse_dict = {}
    for k in mydict.keys():
        if mydict[k] in inverse_dict:
            raise "Cannot Construct inverse dictionary, as function not one-one"
        inverse_dict[mydict[k]] = k
    return inverse_dict

In [ ]:
distmult_dump = read_pkl('../symbolic/distmult_dump_norm.pkl')
test_data = read_data('../data/fb15k/test.txt')
mapped_test_data = map_data(test_data,distmult_dump['entity_to_id'],distmult_dump['relation_to_id'])
entity_names = read_entity_names('../data/fb15k/entity_mid_name_type_typeid.txt')

In [ ]:
entity_inverse_map = get_inverse_dict(distmult_dump['entity_to_id'])
relation_inverse_map = get_inverse_dict(distmult_dump['relation_to_id'])

In [ ]:
import template_builder

In [ ]:
_,template_objs = template_builder.template_obj_builder("../data/fb15k/","../symbolic/distmult_dump_norm.pkl",
                                                      "logs/25-10-2018-1930/",None,"distmult",[5],True)

In [ ]:
def get_header(t_type):
    if(t_type==1 or t_type==2):
        msg = 'most freq for '+('relation' if t_type==1 else 'entity')
        return ['e1','r','e2','my_score',msg,'best_score','z_score']
    elif(t_type==3 or t_type==4):
        msg = 'best_'+('relation' if t_type==3 else 'entity')        
        return ['e1','r','e2','my_score','my_'+msg,'best_score','best_answer','best_answer_'+msg,'z_score']
    elif(t_type==5):
        return ['e1','r','e2','my_score','my_best_e','my_best_r','best_score',
                'best_answer','best_answer_best_e','best_answer_best_r','z_score']
    else:
        raise "Invalid Type"

In [ ]:
def get_word_exp(exp,t_type,entity_id_inverse_map,relation_id_inverse_map,entity_name_map):
    exp_word_list = []
    if(t_type==1 or t_type==2):
        exp_word_list.append(exp[0])
        exp_word_list.append(exp[1])
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[2],"None"),"None"))
        exp_word_list.append(exp[3])
    elif(t_type==3):
        exp_word_list.append(exp[0])
        exp_word_list.append(relation_inverse_map.get(exp[1],"None"))
        exp_word_list.append(exp[2])
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[3],"None"),"None"))
        exp_word_list.append(relation_inverse_map.get(exp[4],"None"))
        exp_word_list.append(exp[5])
    elif(t_type==4):
        exp_word_list.append(exp[0])
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[1],"None"),"None"))
        exp_word_list.append(exp[2])
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[3],"None"),"None"))
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[4],"None"),"None"))
        exp_word_list.append(exp[5])
    elif(t_type==5):
        exp_word_list.append(exp[0])
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[1][0],"None"),"None"))
        exp_word_list.append(relation_inverse_map.get(exp[1][1],"None"))
        exp_word_list.append(exp[2])
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[3],"None"),"None"))
        exp_word_list.append(entity_name_map.get(entity_id_inverse_map.get(exp[4][0],"None"),"None"))
        exp_word_list.append(relation_inverse_map.get(exp[4][1],"None"))
        exp_word_list.append(exp[5])
    else:
        raise "Invalid Template ID"
    
    return exp_word_list

In [ ]:
def generate_exp(data,template_obj_list,entity_id_inverse_map,relation_id_inverse_map,entity_name_map):
    word_exps = {}
    for t in template_obj_list:
        t_type = int(type(t).__name__[-1])
        exp_list = [get_header(t_type)]
        for triple in data:
            temp_list = []
            temp_list.append(entity_name_map[entity_id_inverse_map[triple[0]]])
            temp_list.append(relation_id_inverse_map[triple[1]])
            temp_list.append(entity_name_map[entity_id_inverse_map[triple[2]]])          
            exp = t.get_explanation(triple)
            temp_list.extend(get_word_exp(exp,t_type,entity_id_inverse_map,relation_id_inverse_map,entity_name_map))
            exp_list.append(temp_list)
        word_exps[t_type] = exp_list
    return word_exps

In [ ]:
word_exps = generate_exp(mapped_test_data,template_objs,entity_inverse_map,relation_inverse_map,entity_names)

In [ ]:
def write_word_exps(word_exps,output_root_path):
    for t_id in word_exps.keys():
        with open(os.path.join(output_root_path,str(t_id)+".txt"),"w") as f:
            for el in word_exps[t_id]:
                f.write(' ; '.join(map(str,el))+'\n')

In [ ]:
write_word_exps(word_exps,"logs/exp/")